In [2]:
import pandas as pd
from tabulate import tabulate

def format_mean_std(row, metric):
    mean_val = row[(metric, 'mean')]
    std_val = row[(metric, 'std')]
    return f"{mean_val:.3f} ± {std_val:.3f}"


def get_stat_df(filename, verbose=True):
    # Sample loading of your CSV - this would be replaced with your actual loading
    result_df = pd.read_csv(filename)

    # metrics = ['mcc', 'sensitivity', 'specificity', 'accuracy', 'precision', 'micro_auroc']
    metrics = ['mcc', 'sensitivity', 'precision', 'micro_auroc']
    grouped = result_df.groupby('model_key').agg({
        k: ['mean', 'std'] for k in metrics
    })

    record_counts = result_df.groupby('model_key').size()

    for metric in metrics:
        grouped[f'{metric}_formatted'] = grouped.apply(lambda row: format_mean_std(row, metric), axis=1)


    # Add the record count column to the grouped dataframe
    grouped['record_count'] = record_counts.values

    formatted_df = grouped[[k + '_formatted' for k in metrics] + ['record_count']]
    formatted_df.columns = metrics + ['record_count']
    formatted_df = formatted_df.sort_values(metrics[0], ascending=True)
    '''
    ESM:
    2: esm-t33-pretrained
    4: esm-33-gearnet-resiboost	-> 2번하고 비교헀을 때: gearnet을 추가하는 게 더 낫다. resiboost를 추가하는 것도 좋다.
    9: esm-33-gearnet-pretrained-freezelm-ensemble -> 4번하고 비교했을 때 pretrain된 거니까 좋다.


    '''
    formatted_df = formatted_df.reset_index()
    
    if verbose:
        markdown_table = tabulate(formatted_df[['model_key'] + metrics] , headers='keys', tablefmt='pipe', showindex=False)
        print(markdown_table)

    return formatted_df



In [3]:
'''
get_stat_df_summary:

'''

def get_stat_df_summary(filename, groups, verbose=True):
    '''
    - can group some model_key together, and select the model with maximum mcc
    - groups is a dict: {group_name: [model_key1, model_key2, ...]}
    '''
    stat_df = get_stat_df(filename, verbose=False)
    
    new_df = pd.DataFrame()
    
    for group_name, model_keys in groups.items():
        group_df = stat_df[stat_df['model_key'].isin(model_keys)]
        if group_df.shape[0] == 0:
            continue
        group_df = group_df.sort_values('mcc', ascending=False)
        group_df = group_df.head(1).copy().reset_index(drop=True)
        group_df.insert(0, 'group_name', group_name)
        new_df = pd.concat([new_df, group_df])
        
    new_df = new_df.sort_values('mcc', ascending=True).reset_index(drop=True)
    
    metrics = ['mcc', 'sensitivity', 'precision', 'micro_auroc']
    if verbose:
        markdown_table = tabulate(new_df[['group_name', 'model_key'] + metrics] , headers='keys', tablefmt='pipe', showindex=False)
        print(markdown_table)
    
    return new_df
        
    
groups = {
    'ESM': ['esm-t33'],
    'ESM + Pretrained': ['esm-t33-pretrained', 'esm-t33-pretrained-freezelm'],
    'ESM + GearNet': ['esm-33-gearnet'],
    'ESM + GearNet + Ensemble': ['esm-33-gearnet-ensemble'],
    'ESM + GearNet + Pretrained + Ensemble': ['esm-33-gearnet-pretrained-ensemble'],
    'ESM + GearNet + Pretrained': ['esm-33-gearnet-pretrained', 'esm-33-gearnet-pretrained-freezelm', 'esm-33-gearnet-pretrained-freezeall'],
    'ESM + GearNet + Resiboost': [f'esm-33-gearnet-rboost{ratio:02d}' for ratio in [5, 10, 25, 50]] + [f'esm-33-gearnet-rboost{ratio:02d}-b8' for ratio in [5, 10, 25, 50]],
    'ESM + GearNet + Pretrained + Resiboost': [f'esm-33-gearnet-pretrained-freezelm-rboost{ratio:02d}' for ratio in [5, 10, 25, 50]] + 
                                              [f'esm-33-gearnet-pretrained-rboost{ratio:02d}' for ratio in [10, 50]],
    'ESM + GearNet + Pretrained + Ensemble': [f'esm-33-gearnet-pretrained-ensemble', 'esm-33-gearnet-pretrained-freezelm-ensemble'],
}


get_stat_df_summary('record.csv', groups=groups)


| group_name                 | model_key                   | mcc           | sensitivity   | precision     | micro_auroc   |
|:---------------------------|:----------------------------|:--------------|:--------------|:--------------|:--------------|
| ESM                        | esm-t33                     | 0.662 ± 0.032 | 0.702 ± 0.043 | 0.675 ± 0.079 | 0.940 ± 0.023 |
| ESM + Pretrained           | esm-t33-pretrained-freezelm | 0.674 ± 0.019 | 0.684 ± 0.028 | 0.709 ± 0.021 | 0.857 ± 0.025 |
| ESM + GearNet + Ensemble   | esm-33-gearnet-ensemble     | 0.709 ± 0.011 | 0.709 ± 0.009 | 0.749 ± 0.017 | 0.913 ± 0.006 |
| ESM + GearNet              | esm-33-gearnet              | 0.719 ± 0.015 | 0.725 ± 0.018 | 0.752 ± 0.026 | 0.910 ± 0.018 |
| ESM + GearNet + Pretrained | esm-33-gearnet-pretrained   | 0.721 ± 0.016 | 0.736 ± 0.021 | 0.743 ± 0.019 | 0.893 ± 0.022 |


,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,ESM,esm-t33,0.662 ± 0.032,0.702 ± 0.043,0.675 ± 0.079,0.940 ± 0.023,5
1,ESM + Pretrained,esm-t33-pretrained-freezelm,0.674 ± 0.019,0.684 ± 0.028,0.709 ± 0.021,0.857 ± 0.025,5
2,ESM + GearNet + Ensemble,esm-33-gearnet-ensemble,0.709 ± 0.011,0.709 ± 0.009,0.749 ± 0.017,0.913 ± 0.006,5
3,ESM + GearNet,esm-33-gearnet,0.719 ± 0.015,0.725 ± 0.018,0.752 ± 0.026,0.910 ± 0.018,5
4,ESM + GearNet + Pretrained,esm-33-gearnet-pretrained,0.721 ± 0.016,0.736 ± 0.021,0.743 ± 0.019,0.893 ± 0.022,5


In [4]:
get_stat_df('imatinib.csv')
print('--------GROUPED VERSION--------')
get_stat_df_summary('imatinib.csv', groups=groups)

| model_key                                   | mcc           | sensitivity   | precision     | micro_auroc   |
|:--------------------------------------------|:--------------|:--------------|:--------------|:--------------|
| esm-33-gearnet-pretrained-freezeall         | 0.332 ± 0.083 | 0.293 ± 0.057 | 0.482 ± 0.202 | 0.769 ± 0.013 |
| esm-33-gearnet                              | 0.353 ± 0.087 | 0.389 ± 0.107 | 0.437 ± 0.186 | 0.743 ± 0.040 |
| esm-33-gearnet-pretrained-freezelm-rboost05 | 0.376 ± 0.025 | 0.509 ± 0.014 | 0.346 ± 0.041 | 0.790 ± 0.009 |
| esm-33-gearnet-pretrained-freezelm          | 0.390 ± 0.059 | 0.435 ± 0.061 | 0.430 ± 0.134 | 0.803 ± 0.021 |
| esm-33-gearnet-pretrained-freezelm-ensemble | 0.403 ± 0.058 | 0.343 ± 0.041 | 0.541 ± 0.099 | 0.806 ± 0.016 |
| esm-t33                                     | 0.429 ± 0.028 | 0.480 ± 0.030 | 0.449 ± 0.062 | 0.826 ± 0.031 |
| esm-33-gearnet-pretrained-freezelm-rboost10 | 0.431 ± 0.049 | 0.443 ± 0.021 | 0.486 ± 0.089 | 0.793 ± 

,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,ESM + GearNet,esm-33-gearnet,0.353 ± 0.087,0.389 ± 0.107,0.437 ± 0.186,0.743 ± 0.040,5
1,ESM,esm-t33,0.429 ± 0.028,0.480 ± 0.030,0.449 ± 0.062,0.826 ± 0.031,5
2,ESM + GearNet + Ensemble,esm-33-gearnet-ensemble,0.452 ± 0.036,0.302 ± 0.047,0.739 ± 0.069,0.790 ± 0.013,5
3,ESM + GearNet + Pretrained,esm-33-gearnet-pretrained,0.479 ± 0.020,0.385 ± 0.045,0.656 ± 0.041,0.760 ± 0.026,5
4,ESM + GearNet + Pretrained + Ensemble,esm-33-gearnet-pretrained-ensemble,0.482 ± 0.021,0.322 ± 0.028,0.775 ± 0.015,0.789 ± 0.012,5
5,ESM + Pretrained,esm-t33-pretrained,0.518 ± 0.017,0.424 ± 0.020,0.687 ± 0.044,0.813 ± 0.018,5
6,ESM + GearNet + Resiboost,esm-33-gearnet-rboost50,0.522 ± 0.017,0.372 ± 0.032,0.788 ± 0.078,0.782 ± 0.020,5
7,ESM + GearNet + Pretrained + Resiboost,esm-33-gearnet-pretrained-rboost50,0.523 ± 0.022,0.372 ± 0.014,0.786 ± 0.044,0.786 ± 0.011,5


In [5]:

get_stat_df('dasatinib.csv')
print('--------GROUPED VERSION--------')
get_stat_df_summary('dasatinib.csv', groups=groups)

| model_key                                   | mcc           | sensitivity   | precision     | micro_auroc   |
|:--------------------------------------------|:--------------|:--------------|:--------------|:--------------|
| esm-33-gearnet-pretrained-freezeall         | 0.628 ± 0.077 | 0.514 ± 0.126 | 0.833 ± 0.077 | 0.884 ± 0.014 |
| esm-33-gearnet                              | 0.635 ± 0.059 | 0.586 ± 0.070 | 0.744 ± 0.089 | 0.933 ± 0.015 |
| esm-33-gearnet-pretrained-freezelm-ensemble | 0.657 ± 0.020 | 0.506 ± 0.036 | 0.901 ± 0.032 | 0.927 ± 0.029 |
| esm-33-gearnet-ensemble                     | 0.661 ± 0.020 | 0.509 ± 0.036 | 0.904 ± 0.014 | 0.949 ± 0.012 |
| esm-33-gearnet-pretrained-freezelm          | 0.661 ± 0.026 | 0.514 ± 0.040 | 0.897 ± 0.035 | 0.926 ± 0.019 |
| esm-t33-pretrained-freezelm                 | 0.669 ± 0.050 | 0.571 ± 0.089 | 0.834 ± 0.048 | 0.893 ± 0.005 |
| esm-33-gearnet-pretrained-ensemble          | 0.679 ± 0.024 | 0.531 ± 0.031 | 0.912 ± 0.014 | 0.947 ± 

,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,ESM + GearNet,esm-33-gearnet,0.635 ± 0.059,0.586 ± 0.070,0.744 ± 0.089,0.933 ± 0.015,5
1,ESM + GearNet + Ensemble,esm-33-gearnet-ensemble,0.661 ± 0.020,0.509 ± 0.036,0.904 ± 0.014,0.949 ± 0.012,5
2,ESM + GearNet + Pretrained + Ensemble,esm-33-gearnet-pretrained-ensemble,0.679 ± 0.024,0.531 ± 0.031,0.912 ± 0.014,0.947 ± 0.008,5
3,ESM + GearNet + Pretrained,esm-33-gearnet-pretrained,0.686 ± 0.044,0.569 ± 0.065,0.873 ± 0.027,0.929 ± 0.036,5
4,ESM + Pretrained,esm-t33-pretrained,0.704 ± 0.023,0.606 ± 0.026,0.859 ± 0.033,0.935 ± 0.028,5
5,ESM,esm-t33,0.714 ± 0.021,0.597 ± 0.031,0.894 ± 0.030,0.954 ± 0.017,5
6,ESM + GearNet + Resiboost,esm-33-gearnet-rboost50,0.716 ± 0.018,0.583 ± 0.028,0.919 ± 0.011,0.946 ± 0.009,5
7,ESM + GearNet + Pretrained + Resiboost,esm-33-gearnet-pretrained-freezelm-rboost10,0.744 ± 0.031,0.666 ± 0.031,0.870 ± 0.059,0.931 ± 0.022,5


In [6]:
get_stat_df('bosutinib.csv')
print('--------GROUPED VERSION--------')
get_stat_df_summary('bosutinib.csv', groups=groups)

| model_key                                   | mcc           | sensitivity   | precision     | micro_auroc   |
|:--------------------------------------------|:--------------|:--------------|:--------------|:--------------|
| esm-33-gearnet-pretrained-freezeall         | 0.683 ± 0.032 | 0.647 ± 0.077 | 0.777 ± 0.113 | 0.948 ± 0.012 |
| esm-t33-pretrained-freezelm                 | 0.687 ± 0.019 | 0.640 ± 0.085 | 0.794 ± 0.130 | 0.963 ± 0.005 |
| esm-t33-rboost50                            | 0.731 ± 0.018 | 0.620 ± 0.027 | 0.899 ± 0.018 | 0.952 ± 0.015 |
| esm-t33-pretrained-rboost50                 | 0.735 ± 0.019 | 0.620 ± 0.014 | 0.908 ± 0.036 | 0.972 ± 0.007 |
| esm-33-gearnet-pretrained-ensemble          | 0.741 ± 0.021 | 0.643 ± 0.040 | 0.890 ± 0.012 | 0.955 ± 0.020 |
| esm-33-gearnet-pretrained                   | 0.742 ± 0.031 | 0.663 ± 0.065 | 0.868 ± 0.032 | 0.953 ± 0.022 |
| esm-t33-pretrained                          | 0.746 ± 0.031 | 0.667 ± 0.026 | 0.870 ± 0.036 | 0.964 ± 

,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,ESM + Pretrained,esm-t33-pretrained,0.746 ± 0.031,0.667 ± 0.026,0.870 ± 0.036,0.964 ± 0.006,5
1,ESM + GearNet,esm-33-gearnet,0.749 ± 0.041,0.677 ± 0.066,0.866 ± 0.043,0.958 ± 0.021,5
2,ESM + GearNet + Pretrained + Ensemble,esm-33-gearnet-pretrained-freezelm-ensemble,0.753 ± 0.023,0.667 ± 0.041,0.886 ± 0.022,0.963 ± 0.010,5
3,ESM,esm-t33,0.754 ± 0.019,0.683 ± 0.037,0.866 ± 0.027,0.928 ± 0.034,5
4,ESM + GearNet + Ensemble,esm-33-gearnet-ensemble,0.756 ± 0.018,0.653 ± 0.027,0.907 ± 0.004,0.964 ± 0.013,5
5,ESM + GearNet + Pretrained,esm-33-gearnet-pretrained-freezelm,0.768 ± 0.025,0.740 ± 0.074,0.833 ± 0.046,0.963 ± 0.010,5
6,ESM + GearNet + Resiboost,esm-33-gearnet-rboost50,0.777 ± 0.015,0.697 ± 0.022,0.897 ± 0.016,0.956 ± 0.014,5
7,ESM + GearNet + Pretrained + Resiboost,esm-33-gearnet-pretrained-freezelm-rboost10,0.801 ± 0.031,0.803 ± 0.036,0.828 ± 0.048,0.954 ± 0.016,5


In [7]:
get_stat_df('methotrexate.csv')
print('--------GROUPED VERSION--------')
get_stat_df_summary('methotrexate.csv', groups=groups)

| model_key                                   | mcc           | sensitivity   | precision     | micro_auroc   |
|:--------------------------------------------|:--------------|:--------------|:--------------|:--------------|
| esm-33-gearnet-pretrained-freezeall         | 0.218 ± 0.041 | 0.242 ± 0.073 | 0.269 ± 0.084 | 0.705 ± 0.056 |
| esm-33-gearnet                              | 0.441 ± 0.040 | 0.329 ± 0.059 | 0.650 ± 0.120 | 0.716 ± 0.062 |
| esm-33-gearnet-ensemble                     | 0.456 ± 0.051 | 0.300 ± 0.064 | 0.741 ± 0.008 | 0.724 ± 0.068 |
| esm-33-gearnet-pretrained                   | 0.456 ± 0.053 | 0.321 ± 0.059 | 0.696 ± 0.072 | 0.745 ± 0.079 |
| esm-33-gearnet-pretrained-ensemble          | 0.466 ± 0.058 | 0.300 ± 0.061 | 0.768 ± 0.031 | 0.770 ± 0.070 |
| esm-t33-pretrained-freezelm                 | 0.469 ± 0.050 | 0.468 ± 0.143 | 0.538 ± 0.090 | 0.819 ± 0.063 |
| esm-33-gearnet-pretrained-freezelm-ensemble | 0.483 ± 0.062 | 0.345 ± 0.065 | 0.718 ± 0.059 | 0.802 ± 

,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,ESM + GearNet,esm-33-gearnet,0.441 ± 0.040,0.329 ± 0.059,0.650 ± 0.120,0.716 ± 0.062,5
1,ESM + GearNet + Ensemble,esm-33-gearnet-ensemble,0.456 ± 0.051,0.300 ± 0.064,0.741 ± 0.008,0.724 ± 0.068,5
2,ESM + GearNet + Pretrained + Ensemble,esm-33-gearnet-pretrained-freezelm-ensemble,0.483 ± 0.062,0.345 ± 0.065,0.718 ± 0.059,0.802 ± 0.080,5
3,ESM + GearNet + Pretrained,esm-33-gearnet-pretrained-freezelm,0.495 ± 0.079,0.395 ± 0.120,0.674 ± 0.049,0.797 ± 0.086,5
4,ESM + Pretrained,esm-t33-pretrained,0.514 ± 0.091,0.453 ± 0.142,0.643 ± 0.116,0.871 ± 0.049,5
5,ESM + GearNet + Resiboost,esm-33-gearnet-rboost50,0.525 ± 0.059,0.390 ± 0.078,0.750 ± 0.015,0.784 ± 0.076,5
6,ESM,esm-t33,0.525 ± 0.079,0.482 ± 0.131,0.630 ± 0.102,0.881 ± 0.049,5
7,ESM + GearNet + Pretrained + Resiboost,esm-33-gearnet-pretrained-rboost10,0.559 ± 0.089,0.473 ± 0.109,0.700 ± 0.055,0.810 ± 0.072,5


In [8]:
get_stat_df('atpbind3d.csv')
print('--------GROUPED VERSION--------')
get_stat_df_summary('atpbind3d.csv', groups=groups)

| model_key                  | mcc           | sensitivity   | precision     | micro_auroc   |
|:---------------------------|:--------------|:--------------|:--------------|:--------------|
| esm-33-gearnet             | 0.566 ± 0.065 | 0.510 ± 0.043 | 0.676 ± 0.109 | 0.914 ± 0.024 |
| esm-33-gearnet-ensemble    | 0.656 ± 0.023 | 0.565 ± 0.026 | 0.795 ± 0.033 | 0.942 ± 0.007 |
| esm-33-gearnet-b8          | 0.672 ± 0.013 | 0.633 ± 0.017 | 0.748 ± 0.019 | 0.927 ± 0.006 |
| esm-33-gearnet-rboost50    | 0.674 ± 0.012 | 0.583 ± 0.014 | 0.812 ± 0.016 | 0.935 ± 0.005 |
| esm-33-gearnet-rboost10    | 0.682 ± 0.009 | 0.661 ± 0.026 | 0.737 ± 0.028 | 0.937 ± 0.006 |
| esm-33-gearnet-rboost25-b8 | 0.683 ± 0.011 | 0.605 ± 0.015 | 0.803 ± 0.017 | 0.912 ± 0.006 |
| esm-33-gearnet-rboost10-b8 | 0.686 ± 0.004 | 0.651 ± 0.028 | 0.756 ± 0.023 | 0.916 ± 0.008 |
| esm-33-gearnet-rboost05-b8 | 0.689 ± 0.007 | 0.688 ± 0.010 | 0.723 ± 0.009 | 0.924 ± 0.006 |
--------GROUPED VERSION--------
| group_name      

,group_name,model_key,mcc,sensitivity,precision,micro_auroc,record_count
0,ESM + GearNet,esm-33-gearnet,0.566 ± 0.065,0.510 ± 0.043,0.676 ± 0.109,0.914 ± 0.024,5
1,ESM + GearNet + Ensemble,esm-33-gearnet-ensemble,0.656 ± 0.023,0.565 ± 0.026,0.795 ± 0.033,0.942 ± 0.007,10
2,ESM + GearNet + Resiboost,esm-33-gearnet-rboost05-b8,0.689 ± 0.007,0.688 ± 0.010,0.723 ± 0.009,0.924 ± 0.006,5
